In this worksheet we construct a program `orbifold_polytope`. The input is a polyhedron $P$ of dimension $3$, homeomorphic to a ball and four outputs:
- $K_1:=\pi_1(Z)$ where $Z$ is the intersection of quadrics associated to $P$.
- $h_1:K_1\to G_1$, the inclusion associated to the orbifold covering morphism $Z\to P$.
- $G_1:=\pi_1^{\text{orb}}(P)$, the orbifold fundamental group of $P$.
- $K_2:=\pi_1(\check{Z})$ where $\check{Z}:=Z\setminus\text{Sing}(Z)$ and $\check{P}$ is the complement in $P$ of the non-simple vertices.
- $h_2:K_2\to G_2$, the inclusion associated to the orbifold covering morphism $\check{Z}\to\check{P}$.
- $G_2:=\pi_1^{\text{orb}}(\check{P})$, the orbifold fundamental group of $\check{P}$.

The following cell defines the needed functions.

In [1]:
from sage.groups.finitely_presented import wrap_FpGroup

def nucleos(lf, rlk, rlk1, debug=False):
    H = FreeGroup(lf) / rlk1
    G = FreeGroup(lf) / rlk
    h = G.hom(codomain=H, im_gens=H.gens())
    K = libgap(h.kernel())
    homfp = K.IsomorphismFpGroup()
    Kgap = homfp.Range()
    homsimp = Kgap.IsomorphismSimplifiedFpGroup()
    Kgapsimp = homsimp.Range()
    preimages = [homfp.PreImagesRepresentative(homsimp.PreImagesRepresentative(a))
                 for a in Kgapsimp.GeneratorsOfGroup()]
    preimages_word = [a.UnderlyingElement().TietzeWordAbstractWord()
                      for a in preimages]
    K = wrap_FpGroup(Kgapsimp)
    return (K, preimages_word, G)

def orbifold_polytope(P, debug=False):
    V = P.vertices()    
    F = P.faces(2)
    lf = len(F)
    if debug:
        print("Vertices: ", len(V))
    V0 = []
    for v in V:
        A = [f for f in F if v in f.vertices()]
        if len(A) <= 3:
            V0.append(v)
    rlk = [2 * (j, ) for j in [1 .. lf]]
    rlk0 = [r for r in rlk]
    rlk1 = [r for r in rlk]
    for (j1, f1), (j2, f2) in Combinations(enumerate(F), 2):
        r0 = (j1 + 1, j2 + 1, -j1 - 1, -j2 - 1)
        rlk1.append(r0)
        A = [v for v in f1.vertices() if v in f2.vertices()]
        if A:
            rlk.append(r0)
        if len(A) == 2 or (len(A) == 1 and A[0] in V0):
            rlk0.append(r0)
    K, h, G = nucleos(lf, rlk, rlk1, debug=debug)
    if rlk0 != rlk:
        K0, h0, G0 = nucleos(lf, rlk0, rlk1, debug=debug)
    else:
        K0, h0, G0 = (K, h, G)
    return (K, h, G, K0, h0, G0)

We start with a cube.

In [22]:
P = polytopes.cube()
P.show(frame=False)
K, h, G, K0, h0, G0 = orbifold_polytope(P)

Graphics3d Object

Since the polytope is simple, the two triples of output coincide

In [3]:
h == h0

True

Next cell shows that $\pi_1(Z)=\mathbb{Z}^3$.

In [4]:
K

Finitely presented group < F1, F2, F3 | F2*F1*F2^-1*F1^-1, F3*F1*F3^-1*F1^-1, F3*F2*F3^-1*F2^-1 >

Let us apply it to the tetrahedron.

In [23]:
P = polytopes.tetrahedron()
P.show(frame=False)
K, h, G, K0, h0, G0 = orbifold_polytope(P)
h == h0

Graphics3d Object

True

The intersection $Z$ is homeomorphic to $\mathbb{S}^3$, i.e., simply connected.

In [7]:
K

Finitely presented group <  |  >

Next polytope is a pyramid over an $m$-polygon. We apply it to $m=4$. 

In [24]:
m = 4
P = polytopes.regular_polygon(m).pyramid()
P.plot(frame=False)

Graphics3d Object

La intersección $Z$ is homeomorphic to the suspension of a torus, hence simpliy connected, while $\check{Z}$ has the homotopy type of a torus and its fundamental group is $\mathbb{Z}^2$.

In [14]:
K, h, G, K0, h0, G0 = orbifold_polytope(P)
show(K)
show(K0)

Finitely presented group <  |  >

Finitely presented group < F1, F2 | F2*F1*F2^-1*F1^-1 >

We study now the prism over an $m$-polygon (simple polytope), $m=3$.

In [25]:
m = 3
P = polytopes.regular_polygon(m).prism()
P.show(frame=False)
K, h, G, K0, h0, G0 = orbifold_polytope(P)
h == h0

Graphics3d Object

True

The intersección is homeomorphic to $\mathbb{S}^2\times\mathbb{S}^1$, i.e, its fundamental group is $\mathbb{Z}$.

In [18]:
K

Finitely presented group < F1 |  >

Next examples are bipyrámids over an $m$-polygon, which are not simple. We apply for the triangular one.

In [26]:
m = 3
P = polytopes.regular_polygon(3).bipyramid()
P.plot(frame=False)

Graphics3d Object

As we state in the paper, $\pi_1(Z)$ is trivial.

In [14]:
K, h, G, K0, h0, G0 = orbifold_polytope(P)
show(K)

Finitely presented group <  |  >

The following cells give information about $\pi_1(\check{Z})$. The number of generators:

In [15]:
print(K0.ngens())

12


A look about the relations. For each tuple $(\varepsilon_1 a_1,\dots,\varepsilon_r a_r)$, $1\leq a_i\leq 12$, $\varepsilon_i=\pm 1$, we get the relation:
$$
x_{a_1}^{\varepsilon_1}\cdot\ldots\cdot x_{a_r}^{\varepsilon_r}.
$$

In [16]:
for r in K0.relations():
    print(r.Tietze())

(-9, -7, 9, 7)
(3, 1, -3, -1)
(5, 4, -5, -4)
(8, 6, -8, -6)
(3, 5, -8, 2, 8, -5, -3, -2)
(-5, 11, -8, 12, -11, 5, -12, 8)
(11, -10, 6, -11, 9, -6, 10, -9)
(12, -10, 4, -12, 7, -4, 10, -7)
(2, 7, -4, 1, 9, -6, 4, -7, -2, 6, -9, -1)
(2, 12, -4, 6, -11, -1, 4, 5, -12, -2, 8, -5, 1, 11, -8, -6)
(3, 11, -8, 7, -10, 4, -7, -2, 6, 8, -11, -3, 9, -6, 2, 7, -4, 10, -9, -7)


We deduce that the abelianization of $\pi_1(\check{Z})$ is $\mathbb{Z}^{12}$.

In [17]:
L = K0.abelian_invariants()
Set(L), len(L)

({0}, 12)

Next polytope is one of the smoothings of the triangular bipyramid, the pentagonal *book*.

In [27]:
P0 = vector([0, 0, 0])
P1 = vector([1, 0, 0])
P2 = vector([2, 1, 0])
P3 = vector([1 / 2, 2, 0])
P4 = vector([-1, 1, 0])
P5 = vector([2, 1, 1])
P6 = vector([1 / 2, 2, 2])
P7 = vector([-1, 1, 1])
P = Polyhedron([P0, P1, P2, P3, P4, P5, P6, P7])
P.show(frame=False)
K, h, G, K0, h0, G0 = orbifold_polytope(P)
h == h0

Graphics3d Object

True

The intersection $Z$ is homeomorphic to $\#^5\mathbb{S}^2\times\mathbb{S}^1$, i.e., its fundamental group is free of rank $5$.

In [31]:
K

Finitely presented group < F1, F2, F3, F4, F5 |  >

Let us take now the octahedron. Sometimes, this computation takes a while, but stop and recompute its fast.

In [29]:
P = polytopes.octahedron()
K, h, G, K0, h0, G0 = orbifold_polytope(P)
K

Finitely presented group < F1, F2, F3, F4 | F2*F1*F2^-1*F1^-1, F3*F1*F3^-1*F1^-1, F3*F2*F3^-1*F2^-1, F4*F1*F4^-1*F1^-1, F4*F2*F4^-1*F2^-1, F4*F3*F4^-1*F3^-1 >

The following cell exhibits that the fundamental group of $Z$ is $\mathbb{Z}^4$.

In [35]:
for r in K.relations():
    print(r.Tietze())

(2, 1, -2, -1)
(3, 1, -3, -1)
(3, 2, -3, -2)
(4, 1, -4, -1)
(4, 2, -4, -2)
(4, 3, -4, -3)


The fundamental group of $\pi_1(\check{Z})$ is quite complicated; its abelianization is $\mathbb{Z}^{100}$.

In [38]:
L = K0.abelian_invariants()
Set(L), len(L)

({0}, 100)